<a href="https://colab.research.google.com/github/BakJungHo/PaCO/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B81/%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%8F%99%EC%9D%BC%EC%97%85%EC%A2%85PER%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

네이버금융 재무제표 크롤링(1)
==

In [228]:
statement

,종목코드,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족
0,095570,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0
1,006840,AK홀딩스,NaN,8.54,NaN,0.43,NaN,1.02,0,1,0,0,0,0,0,0,0
2,027410,BGF,7.45,6.23,NaN,0.31,0.04,2.16,1,1,0,0,0,1,0,0,0
3,282330,BGF리테일,21.95,6.23,17.0,4.00,0.18,1.60,0,0,1,0,1,0,0,0,0
4,138930,BNK금융지주,3.18,5.29,3.0,0.27,0.08,7.25,1,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,005870,휴니드,9.88,31.16,NaN,0.90,0.09,NaN,1,1,0,1,0,0,0,0,0
816,079980,휴비스,NaN,4.79,NaN,0.52,NaN,4.20,0,1,0,0,0,1,0,0,0
817,005010,휴스틸,4.68,4.14,NaN,0.29,0.06,3.46,1,1,0,0,0,1,0,0,0
818,000540,흥국화재,3.89,6.72,NaN,0.35,0.09,NaN,1,1,0,1,0,0,0,0,0


In [229]:
df2[['종목명','종목코드']]

,종목명,종목코드
0,AJ네트웍스,095570
13,DB금융투자,016610
18,DL,000210
19,DL건설,001880
20,DL이앤씨,375500
29,GS,078930
32,GS리테일,007070
43,JB금융지주,175330
65,KTis,058860
67,LG,003550


In [230]:
df3[['종목명','종목코드']]

,종목명,종목코드
29,GS,078930
43,JB금융지주,175330
81,LX인터내셔널,001120
176,기아,000270
230,대한제강,084010
244,동국제강,001230
632,코오롱글로벌,003070


In [ ]:
!pip install finance-datareader
!pip install pykrx
!pip install bs4
!pip install html5lib
!pip3 install lxml

In [110]:
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import requests
import re


from datetime import datetime 
from bs4 import BeautifulSoup
from pykrx import stock
%matplotlib inline

In [3]:
# 오늘날 코스피에 상장되어 있는 주식의 이름과 티커 수집 
today = datetime.today().strftime("%Y%m%d") 
ticker_list = stock.get_market_ticker_list(date = today, market="KOSPI") 
symbol_list = [] 
for ticker in ticker_list: 
  symbol = stock.get_market_ticker_name(ticker) 
  symbol_list.append(symbol) 

# 코스피 상장 주식의 개수 확인 
print(len(ticker_list)) 
print(len(symbol_list))

940
940


In [4]:
df_우 = [s for s in symbol_list if s.endswith('우') | s.endswith('우B') | s.endswith('우(전환)')]
res = [i for i in symbol_list if i not in df_우]

ticker = pd.DataFrame(ticker_list)
symbol = pd.DataFrame(symbol_list)
sol = pd.concat([ticker, symbol], axis=1)
sol.columns= ['종목코드', '종목명']
kospi_df = sol[sol['종목명'].isin(res)]
kospi_df.reset_index(inplace=True)
kospi_df.drop('index', axis=1, inplace=True)
kospi_df['종목코드'] = kospi_df['종목코드'].astype('str')
ticker_ks = kospi_df['종목코드'].tolist()
print(len(ticker_ks))
kospi_df

820


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,종목코드,종목명
0,095570,AJ네트웍스
1,006840,AK홀딩스
2,027410,BGF
3,282330,BGF리테일
4,138930,BNK금융지주
...,...,...
815,005870,휴니드
816,079980,휴비스
817,005010,휴스틸
818,000540,흥국화재


In [ ]:
a = []
b = []
c = []
d = []
e = []
f = []
g = []
h = []
i = []
j = []
k = []
l = []
m = []
n = []
o = []
p = []
q = []
r = []
s = []
t = []
u = []
w = []
x = []
y = []
z = []

In [ ]:
ks_o = ['qq', 'ufu']
ks_i = ks_o + ['wuw']
ks_i

['qq', 'ufu', 'wuw']

In [ ]:
import pandas as pd
import requests

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    a.append(yf)

except IndexError:
  a.append(None)
  print(len(a))
  del ticker_ks[:len(a)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    b.append(yf)

except IndexError:
  b.append(None)
  print(len(b))
  del ticker_ks[:len(b)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    c.append(yf)

except IndexError:
  c.append(None)
  print(len(c))
  del ticker_ks[:len(c)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    d.append(yf)

except IndexError:
  d.append(None)
  print(len(d))
  del ticker_ks[:len(d)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    e.append(yf)

except IndexError:
  e.append(None)
  print(len(e))
  del ticker_ks[:len(e)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    f.append(yf)

except IndexError:
  f.append(None)
  print(len(f))
  del ticker_ks[:len(f)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    g.append(yf)

except IndexError:
  g.append(None)
  print(len(g))
  del ticker_ks[:len(g)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    h.append(yf)

except IndexError:
  h.append(None)
  print(len(h))
  del ticker_ks[:len(h)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    i.append(yf)

except IndexError:
  i.append(None)
  print(len(i))
  del ticker_ks[:len(i)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    j.append(yf)

except IndexError:
  j.append(None)
  print(len(j))
  del ticker_ks[:len(j)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    k.append(yf)

except IndexError:
  k.append(None)
  print(len(k))
  del ticker_ks[:len(k)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    l.append(yf)

except IndexError:
  l.append(None)
  print(len(l))
  del ticker_ks[:len(l)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    m.append(yf)

except IndexError:
  m.append(None)
  print(len(m))
  del ticker_ks[:len(m)]

res = a+b+c+d+e+f+g+h+i+j+k+l+m+n+o+p+q+r+s+t+u+w+x+y+z
print(len(res))

26
62
2
24
166
7
14
1
9
8
1
8
5
333


In [ ]:
try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    n.append(yf)

except IndexError:
  n.append(None)
  print(len(n))
  del ticker_ks[:len(n)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    o.append(yf)

except IndexError:
  o.append(None)
  print(len(o))
  del ticker_ks[:len(o)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    p.append(yf)

except IndexError:
  p.append(None)
  print(len(p))
  del ticker_ks[:len(p)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    q.append(yf)

except IndexError:
  q.append(None)
  print(len(q))
  del ticker_ks[:len(q)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    r.append(yf)

except IndexError:
  r.append(None)
  print(len(r))
  del ticker_ks[:len(r)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    s.append(yf)

except IndexError:
  s.append(None)
  print(len(s))
  del ticker_ks[:len(s)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    t.append(yf)

except IndexError:
  t.append(None)
  print(len(t))
  del ticker_ks[:len(t)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    u.append(yf)

except IndexError:
  u.append(None)
  print(len(u))
  del ticker_ks[:len(u)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    w.append(yf)

except IndexError:
  w.append(None)
  print(len(w))
  del ticker_ks[:len(w)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    x.append(yf)

except IndexError:
  x.append(None)
  print(len(x))
  del ticker_ks[:len(x)]

try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    y.append(yf)

except IndexError:
  y.append(None)
  print(len(y))
  del ticker_ks[:len(y)]


try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    z.append(yf)

except IndexError:
  z.append(None)
  print(len(z))
  del ticker_ks[:len(z)]

res = a+b+c+d+e+f+g+h+i+j+k+l+m+n+o+p+q+r+s+t+u+w+x+y+z
print(len(res))

118
1
32
56
9
1
27
45
1
1
64
24
712


In [ ]:
a1 = []
a2 = []
a3 = []
a4 = []
a5 = []

In [ ]:
a1

In [ ]:
len(a1)

107

In [ ]:
try:
  for ticker in ticker_ks:
    URL = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    re = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
    yf = pd.read_html(re.text)[9]
    yf = yf.iloc[0][1]
    a1.append(yf)

except IndexError:
  a1.append(None)
  print(len(a1))
  del ticker_ks[:len(a1)]

KeyboardInterrupt: ignored

In [ ]:
ticker_ks = ticker_ks[1:]

In [ ]:
ticker_kospi = ticker_ks

In [ ]:
ref = res
reg = ref + ['12.87배']
reg

In [ ]:
len(ticker_ks)

108

In [ ]:
len(ticker_kospi)

108

In [ ]:
ticker_ks

In [ ]:
import pandas as pd
dp = pd.DataFrame(sog, columns = ['동일업종PER'])
dp

,동일업종PER
0,10.83배
1,8.54배
2,6.23배
3,6.23배
4,5.29배
...,...
815,31.16배
816,4.79배
817,4.14배
818,6.72배


In [ ]:
dp.to_csv('/content/drive/MyDrive/프로젝트1 데이터/동일업종PER.csv')

In [ ]:
doo = pd.read_csv('/content/drive/MyDrive/프로젝트1 데이터/동일업종PER.csv')
doo

,Unnamed: 0,동일업종PER
0,0,10.83배
1,1,8.54배
2,2,6.23배
3,3,6.23배
4,4,5.29배
...,...,...
815,815,31.16배
816,816,4.79배
817,817,4.14배
818,818,6.72배


In [ ]:
df_.isnull().sum()

동일업종PER    26
dtype: int64

In [ ]:
df_.tail(100)

,동일업종PER
1099,6.23배
1100,9.58배
1101,80.19배
1102,28.73배
1103,18.96배
...,...
1194,3.30배
1195,10.36배
1196,11.99배
1197,12.87배


In [ ]:
sog = reg + a1

In [ ]:
len(sog)

820

In [42]:
a1

NameError: ignored

In [5]:
ind_df = pd.read_csv('/content/drive/MyDrive/프로젝트1 데이터/동일업종PER.csv')
ind_df

,Unnamed: 0,동일업종PER
0,0,10.83배
1,1,8.54배
2,2,6.23배
3,3,6.23배
4,4,5.29배
...,...,...
815,815,31.16배
816,816,4.79배
817,817,4.14배
818,818,6.72배


In [46]:
ind_df.replace('배', '', inplace=True)

In [47]:
ind_df

,Unnamed: 0,동일업종PER
0,0,10.83배
1,1,8.54배
2,2,6.23배
3,3,6.23배
4,4,5.29배
...,...,...
815,815,31.16배
816,816,4.79배
817,817,4.14배
818,818,6.72배


In [7]:
stock_df = pd.read_csv('/content/drive/MyDrive/프로젝트1 데이터/주식데이터.csv')
stock_df

,Unnamed: 0,종목코드,종목명,PER,PBR,배당수익률,추정PER,ROE,PEG
0,0,95570,AJ네트웍스,4.16,0.84,3.96,NaN,0.201923,20.601905
1,1,6840,AK홀딩스,NaN,0.43,1.02,NaN,NaN,NaN
2,2,27410,BGF,7.45,0.31,2.16,NaN,0.041611,179.040323
3,3,282330,BGF리테일,21.95,4.00,1.60,17.0,0.182232,120.450625
4,4,138930,BNK금융지주,3.18,0.27,7.25,3.0,0.084906,37.453333
...,...,...,...,...,...,...,...,...,...
815,815,5870,휴니드,9.88,0.90,NaN,NaN,0.091093,108.460444
816,816,79980,휴비스,NaN,0.52,4.20,NaN,NaN,NaN
817,817,5010,휴스틸,4.68,0.29,3.46,NaN,0.061966,75.525517
818,818,540,흥국화재,3.89,0.35,NaN,NaN,0.089974,43.234571


In [11]:
kos_df = pd.concat([stock_df, ind_df], axis=1)

In [14]:
kos_df = kos_df.drop('Unnamed: 0', axis=1)

In [18]:
kos_df = kos_df.drop(['종목코드', '종목명'], axis=1)

In [27]:
statement = pd.concat([kos_df, kospi_df], axis=1)

In [30]:
statement = statement[['종목코드', '종목명', 'PER', '동일업종PER', '추정PER', 'PBR', 'ROE', '배당수익률']]

In [60]:
statement['동일업종PER'] = statement['동일업종PER'].str.replace("배", "")

In [61]:
statement['동일업종PER']

0      10.83
1       8.54
2       6.23
3       6.23
4       5.29
       ...  
815    31.16
816     4.79
817     4.14
818     6.72
819     3.33
Name: 동일업종PER, Length: 820, dtype: object

In [69]:
statement[['PER', '동일업종PER', '추정PER', 'PBR', 'ROE', '배당수익률']] = statement[['PER', '동일업종PER', '추정PER', 'PBR', 'ROE', '배당수익률']].astype(float)

In [74]:
statement['ROE'] = statement['ROE'].round(2)

In [136]:
import numpy as np

In [188]:
def per(x, y):
  if  x <= y :
    result = 1

  else:
    result = 0

  return result

In [192]:
statement['PER10이하주'] = statement['PER'].apply(lambda x : 1 if x <= 10  else 0)

In [185]:
statement['PBR1이하주'] = statement['PBR'].apply(lambda x : 1 if x <= 1  else 0)

In [186]:
statement['ROE12이상주'] = statement['ROE'].apply(lambda x : 1 if x >= 0.12  else 0)

In [187]:
statement['배당수익률2이상주'] = statement['배당수익률'].apply(lambda x : 1 if x >= 2  else 0)

In [189]:
statement['동일업종대비PER하위주'] = statement.apply(lambda x : per(x['PER'], x['동일업종PER']), axis=1)

In [204]:
statement

,종목코드,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족
0,095570,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0
1,006840,AK홀딩스,NaN,8.54,NaN,0.43,NaN,1.02,0,1,0,0,0,0,0,0,0
2,027410,BGF,7.45,6.23,NaN,0.31,0.04,2.16,1,1,0,0,0,1,0,0,0
3,282330,BGF리테일,21.95,6.23,17.0,4.00,0.18,1.60,0,0,1,0,1,0,0,0,0
4,138930,BNK금융지주,3.18,5.29,3.0,0.27,0.08,7.25,1,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,005870,휴니드,9.88,31.16,NaN,0.90,0.09,NaN,1,1,0,1,0,0,0,0,0
816,079980,휴비스,NaN,4.79,NaN,0.52,NaN,4.20,0,1,0,0,0,1,0,0,0
817,005010,휴스틸,4.68,4.14,NaN,0.29,0.06,3.46,1,1,0,0,0,1,0,0,0
818,000540,흥국화재,3.89,6.72,NaN,0.35,0.09,NaN,1,1,0,1,0,0,0,0,0


In [202]:
def four_pass(a,b,c,d):
  if  a+b+c+d == 4 :
    result = 1

  else:
    result = 0

  return result

def five_pass(a,b,c,d,e):
  if  a+b+c+d+e == 5 :
    result = 1

  else:
    result = 0

  return result

def six_pass(a,b,c,d,e,f):
  if  a+b+c+d+e+f == 6 :
    result = 1

  else:
    result = 0

  return result

In [203]:
statement['2346만족'] = statement.apply(lambda x : four_pass(x['PBR1이하주'], x['ROE12이상주'], x['동일업종대비PER하위주'], x['배당수익률2이상주']), axis=1)
statement['12346만족'] = statement.apply(lambda x : five_pass(x['PER10이하주'], x['PBR1이하주'], x['ROE12이상주'], x['동일업종대비PER하위주'], x['배당수익률2이상주']), axis=1)
statement['123456만족'] = statement.apply(lambda x : six_pass(x['PER10이하주'], x['PBR1이하주'], x['ROE12이상주'], x['동일업종대비PER하위주'], x['PER대비추정PER하위주'], x['배당수익률2이상주']), axis=1)

In [227]:
statement.to_excel('/content/drive/MyDrive/프로젝트1 데이터/네이버_재무제표데이터.xls')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.


In [226]:
statement.to_csv('/content/drive/MyDrive/프로젝트1 데이터/네이버_재무제표데이터.csv')

In [224]:
con2 = statement['12346만족'] == 1
con3 = statement['123456만족'] == 1

df2 = statement.loc[con2, :]
df3 = statement.loc[con3, :]

In [225]:
df2

,종목코드,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족
0,095570,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0
13,016610,DB금융투자,2.16,3.30,NaN,0.27,0.12,8.38,1,1,1,1,0,1,1,1,0
18,000210,DL,1.51,8.48,4.0,0.36,0.24,3.03,1,1,1,1,0,1,1,1,0
19,001880,DL건설,3.15,7.37,5.0,0.58,0.18,3.16,1,1,1,1,0,1,1,1,0
20,375500,DL이앤씨,3.71,7.37,4.0,0.52,0.14,2.69,1,1,1,1,0,1,1,1,0
29,078930,GS,3.01,13.02,2.0,0.42,0.14,4.34,1,1,1,1,1,1,1,1,1
32,007070,GS리테일,2.92,6.21,11.0,0.66,0.23,4.59,1,1,1,1,0,1,1,1,0
43,175330,JB금융지주,3.44,5.29,3.0,0.41,0.12,6.77,1,1,1,1,1,1,1,1,1
65,058860,KTis,3.86,16.35,NaN,0.47,0.12,3.62,1,1,1,1,0,1,1,1,0
67,003550,LG,4.86,7.92,5.0,0.57,0.12,3.71,1,1,1,1,0,1,1,1,0


In [223]:
df3

,종목코드,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족
29,078930,GS,3.01,13.02,2.0,0.42,0.14,4.34,1,1,1,1,1,1,1,1,1
43,175330,JB금융지주,3.44,5.29,3.0,0.41,0.12,6.77,1,1,1,1,1,1,1,1,1
81,001120,LX인터내셔널,4.21,6.59,3.0,0.77,0.18,6.05,1,1,1,1,1,1,1,1,1
176,000270,기아,7.20,7.79,6.0,0.97,0.13,3.55,1,1,1,1,1,1,1,1,1
230,084010,대한제강,3.46,4.14,3.0,0.66,0.19,2.02,1,1,1,1,1,1,1,1,1
244,001230,동국제강,3.04,4.14,3.0,0.61,0.20,2.28,1,1,1,1,1,1,1,1,1
632,003070,코오롱글로벌,3.62,7.37,3.0,0.84,0.23,2.54,1,1,1,1,1,1,1,1,1


In [205]:
statement['2346만족'].value_counts()

0    772
1     48
Name: 2346만족, dtype: int64

In [206]:
statement['12346만족'].value_counts()

0    772
1     48
Name: 12346만족, dtype: int64

In [207]:
statement['123456만족'].value_counts()

0    813
1      7
Name: 123456만족, dtype: int64

In [177]:
statement['동일업종대비PER하위주'].value_counts()

0    495
1    325
Name: 동일업종대비PER하위주, dtype: int64

In [179]:
statement['PER대비추정PER하위주'].value_counts()

0    688
1    132
Name: PER대비추정PER하위주, dtype: int64

In [178]:
statement['배당수익률2이상주'].value_counts()

0    527
1    293
Name: 배당수익률2이상주, dtype: int64

In [176]:
statement['ROE12이상주'].value_counts()

0    632
1    188
Name: ROE12이상주, dtype: int64

In [175]:
statement['PER10이하주'].value_counts()

0    530
1    290
Name: PER10이하주, dtype: int64

In [174]:
statement['PBR1이하주'].value_counts()

1    441
0    379
Name: PBR1이하주, dtype: int64